# Python 知识

## 异步编程
异步编程是一种编程范式，允许程序在等待某些操作完成时继续执行其他任务，而不是被阻塞等待

### 异步编程和多线程、多进程
假设要开一家咖啡店，雇佣了 3 个员工，需要完成 “点单”、“制作咖啡” 和 “打包” 三个任务。
- 多线程
  - 员工都在同一个咖啡店（进程）里工作。这个店里只有一个吧台、一套设备和原料（共享内存和资源）。
  - 员工 A 负责点单，员工 B 负责制作咖啡，员工C 负责打包。
  - 当员工都想用同一台封口机（共享资源）时，他们就必须排队。一个人用的时候，另一个人必须等待。
    - 为了避免混乱，你必须制定严格的规则（锁，Lock），比如“同一时间只有一个人能用封口机”。
- 多进程
  - 开 3 家完全独立的分店（进程）。每家分店都有自己的全套设备和原料（独立的内存空间）。
  - 问题是：如果分店A发明了一个新配方（数据），想告诉分店 B，就必须通过公司的物流系统（进程间通信，IPC）来传递信息。
    - 这个过程比在同一个店里沟通要慢得多，也更麻烦。
- 异步
  - 只雇佣了一个超人服务员（单线程），在一个吧台（一个CPU核心）工作。这个服务员有一个神奇的能力：
    - 他为顾客 A 点单，然后把咖啡放到全自动咖啡机（I/O设备）里制作，这个过程需要2分钟。
    - 他不会傻等，而是立刻去为顾客 B 点单，把他的咖啡也放进另一台机器。
    - 然后他去为顾客 C 点单...
  - 在这期间，他会不断地检查（事件循环）：“A的咖啡好了吗？” “B的好了吗？”
    - 一旦 A 的咖啡机发出“叮”的一声（事件完成通知），他会马上停下手中的活，去给A打包。

| 特性 | 多线程 (Threading) | 多进程 (Multiprocessing) | 异步 (Asyncio) |
| :--- | :--- | :--- | :--- |
| **核心思想** | 多个员工，共享一个吧台 | 开设多家独立分店 | 一个超人服务员，永不空闲 |
| **CPU利用** | 单核（受限于GIL） | **多核并行**（无GIL问题） | 单核 |
| **内存/资源** | 共享内存，开销较小 | **独立内存**，开销大 | 极小（单线程） |
| **数据共享** | 简单（直接读写），但**危险** | 复杂（需IPC），但**安全** | 简单且安全（在单线程内） |
| **切换成本** | 较低（操作系统线程切换） | 高（操作系统进程切换） | **极低**（函数调用级别） |
| **编程模型** | 传统同步，但需处理锁 | 传统同步，但需处理IPC | `async/await`，有学习曲线 |
| **适用场景** | I/O密集型，需共享状态 | **CPU密集型**（科学计算、数据处理） | **高并发I/O密集型**（Web服务、爬虫） |
| **最大痛点** | **GIL**导致无法利用多核CPU | 资源开销大，进程间通信慢 | 无法利用多核，会被CPU密集任务**阻塞** |
| **稳定性** | 较低（一个线程崩溃可能影响整个进程） | 高（进程间隔离） | 较高（单线程无竞争） |

### 生成器
含有 `yield` 关键字的函数就是生成器：
- 第一次 `next()` ：从函数开始执行到第一个 `yield`，返回 `yield` 的值，然后暂停
- 后续 `next()`：从上一个 `yield` 的下一行开始，执行到下一个 `yield`，返回值，再次暂停
- 最后一次：从最后一个 `yield` 后继续执行到函数结束，抛出 `StopIteration` 异常
#### `send` 和 `* = yield ...`
可以向生成器发送值。生成器内部的 `x = yield ...` 相当于两句：
```python
yield ...
x = 接收的内容
```
当外界调用 `生成器.send(*)` 时，生成器内部执行 `yield ...` 后暂停在 `x = *`，于是下一次 `send` 时第一步就是 `x = *`，即生成器内部接收到了外部发送的内容。

`next(生成器)` 等同于 `生成器.send(None)`。

In [ ]:
def echo_generator():
    """回显生成器"""
    value = None
    while True:
        value = yield value
        if value is not None:
            value = f"Echo: {value}"

gen = echo_generator()
next(gen)  # 启动生成器

print(gen.send("Hello"))      # Echo: Hello
print(gen.send("World"))      # Echo: World
print(gen.send(123))          # Echo: 123

#### yield from
可以在生成器内部将该生成器委托给另一个可迭代对象。

In [ ]:
def inner_generator():
    """内部生成器"""
    yield 1
    yield 2
    yield 3

def outer_generator():
    """外部生成器"""
    yield 'start'
    yield from inner_generator()  # 委托给内部生成器
    yield from range(4, 7)        # 委托给range对象
    yield 'end'

# 使用委托生成器
for value in outer_generator():
    print(value)

#### 生成器的返回值
最后一次 `yield` 后，会抛出 `StopIteration` 异常，`return` 的内容会保存在异常信息里。

In [ ]:
def generator_with_return():
    """带返回值的生成器"""
    yield 1
    yield 2
    return "生成器结束"

gen = generator_with_return()
try:
    while True:
        value = next(gen)
        print(f"生成的值: {value}")
except StopIteration as e:
    print(f"生成器返回值: {e.value}")

### 协程

协程（Coroutine）：一种可暂停的函数。
- 使用 `async def` 定义的函数就是一个**协程函数**。
- 调用协程函数不会立即执行它，而是返回一个**协程对象**。

#### `async` 和 `await` 关键字
`async def`：用于定义一个协程函数。

`await`：
- `await` 关键字只能在 `async def` 函数内部使用。
- 后面通常跟着一个可等待对象（Awaitable），比如另一个协程对象、任务（Task）或者 Future。

### `asyncio` 模块：事件循环和任务

事件循环（Event Loop）的作用是
1. 监听和分发**事件**或**任务**。
2. 运行协程，当遇到 await 时暂停它。
3. 在协程暂停期间，去运行其他可以运行的协程。
4. 当暂停的协程所等待的操作完成后，再把它恢复执行。

`asyncio.run(coroutine)` 会自动创建和管理事件循环，并在协程执行完毕后关闭它。


任务（Task）
- 任务是协程的包装器，会被事件**循环**调度执行
- 如果想让多个协程并发（看起来像同时）执行，需要把它们包装成任务（Task）。
- `asyncio.create_task(coroutine)` 可以创建一个任务


In [ ]:
import asyncio
import time

async def fetch_data(source, delay):
    """模拟从不同数据源获取数据"""
    print(f"开始从 {source} 获取数据...")
    await asyncio.sleep(delay)
    print(f"完成从 {source} 获取数据")
    return f"来自 {source} 的数据"

async def main():
    """并发执行任务的主协程"""
    start_time = time.time()
    
    print("开始并发执行任务")
    
    # 创建任务，任务会立刻被事件循环调度执行
    task1 = asyncio.create_task(fetch_data("API 1", 2))
    task2 = asyncio.create_task(fetch_data("数据库", 1))
    
    # 现在，task1和task2已经在后台并发运行了
    print("任务已创建，主协程可以做点别的事情...")
    await asyncio.sleep(0.5)
    print("主协程做完别的事情了，现在等待任务结果")
    
    # 使用await等待任务完成并获取结果
    result1 = await task1
    result2 = await task2
    
    print(f"任务1结果: {result1}")
    print(f"任务2结果: {result2}")
    
    end_time = time.time()
    # 总耗时取决于最长的那个任务
    print(f"总耗时: {end_time - start_time:.2f} 秒")

# asyncio.run(main())
# 在Jupyter中直接使用await，因为Jupyter环境中已经有一个运行中的时间循环
await main()

### `asyncio.gather`
并发执行一组任务，并等待它们全部完成后再继续。

例子：包括两层并发
- 内层并发：对于每一只股票，get_stock_price_data 和 get_recent_news 是并发执行的。
- 外层并发：对 TSLA 的处理和对 AAPL 的处理也是并发执行的。

In [ ]:
import asyncio
import httpx # 一个支持异步的HTTP客户端库
from pygooglenews import GoogleNews

stock_list = ["TSLA", "AAPL"]

async def get_stock_price_data(client, stock):
    """异步获取股票价格"""
    print(f"开始获取 {stock} 的股价...")
    url = f'https://finance.yahoo.com/quote/{stock}'
    response = await client.get(url)
    # 这里省略了解析HTML的复杂代码
    print(f"完成获取 {stock} 的股价")
    return f"{stock} 价格数据"

async def get_recent_news(stock):
    """异步获取新闻"""
    print(f"开始获取 {stock} 的新闻...")
    gn = GoogleNews()
    # 对于不支持async的库，可以使用run_in_executor
    loop = asyncio.get_event_loop()
    search = await loop.run_in_executor(None, gn.search, f"stocks {stock}", '24h')
    print(f"完成获取 {stock} 的新闻")
    return f"{stock} 新闻数据"

async def process_stock_batch(client, stock):
    """处理单个股票的所有数据获取任务"""
    print(f"--- 开始处理批次: {stock} ---")
    
    # 使用gather并发获取价格和新闻
    data, news = await asyncio.gather(
        get_stock_price_data(client, stock),
        get_recent_news(stock)
    )
    
    # 两个都完成后，才执行打印
    print(f"--- 完成处理批次: {stock} ---")
    print(f"  -> 价格: {data}")
    print(f"  -> 新闻: {news}\n")

async def process_all_stocks():
    """处理所有股票"""
    async with httpx.AsyncClient() as client:
        # 为每只股票创建一个处理任务
        tasks = [process_stock_batch(client, stock) for stock in stock_list]
        # 并发执行所有股票的处理任务
        await asyncio.gather(*tasks)

# 运行
await process_all_stocks()

## schedule 库

`schedule` 是一个纯 Python 实现的、无外部依赖的进程内任务调度库。其主要功能是安排周期性任务。

其使用通常分为三步：
- 定义要执行的任务：一个函数 `func`
- 使用 `schedule.every(...).时间单位.do(func, func 的参数)` 来创建 `job`（此时还未运行）
  - `every()` 的作用是创建一个设置了执行间隔的 `job` 对象，这个对象还需要进一步配置时间单位。
  - `do()` 的作用是将具体的任务函数绑定到  `job` 上，完成最终配置。
- 写一个无限循环，在循环中调用 `schedule.run_pending()` 检查是否有 `job` 到期，如果到期则执行该 `job`

In [ ]:
import schedule
import time
import datetime

def greet(name):
    now = datetime.datetime.now().strftime("%H:%M:%S")
    print(f"[{now}] Hello, {name}!")

def report_status():
    now = datetime.datetime.now().strftime("%H:%M:%S")
    print(f"[{now}] 系统状态正常。")

# 每2秒执行一次 greet，并传递参数
schedule.every(2).seconds.do(greet, name="Alice")

# 每5秒执行一次 report_status
schedule.every(5).seconds.do(report_status)

# 每天的 "10:30" 执行 greet
schedule.every().day.at("10:30").do(greet, name="Bob from daily job")

try:
    while True:
        # 检查是否有任务到期需要运行
        schedule.run_pending()
        # 等待1秒，避免CPU空转
        time.sleep(1)
except KeyboardInterrupt:
    print("\n程序被用户终止。")


### 取消任务 `schedule.cancel_job`

In [ ]:
import schedule
import time

def unstoppable_job():
    print("这个任务停不下来！")

job = schedule.every().second.do(unstoppable_job)

start_time = time.time()
while time.time() - start_time < 5.5:
    schedule.run_pending()
    time.sleep(1)

# 运行了大约5秒后，取消这个任务
print("\n准备取消任务...")
schedule.cancel_job(job)
print("任务已取消。")

### 给任务打上标签 `.tag`

In [ ]:
def report_metrics(): print("报告指标...")
def log_activity(): print("记录活动...")
def cleanup_temp_files(): print("清理临时文件...")

# 为任务打上标签
schedule.every().second.do(report_metrics).tag('monitoring', 'reports')
schedule.every(5).seconds.do(log_activity).tag('monitoring')
schedule.every().minute.do(cleanup_temp_files).tag('housekeeping')

# 获取所有带 'monitoring' 标签的任务
monitoring_jobs = schedule.get_jobs('monitoring')
print(f"监控任务: {monitoring_jobs}")

### 查看所有任务 `schedule.jobs`

In [ ]:
print("当前所有已安排的任务:")
for job in schedule.jobs:
    print(job)

### 任务自取消 `schedule.CancelJob`
如果一个任务在某种条件下不想再继续运行，它可以返回 `schedule.CancelJob` 来将自己从调度中移除。

In [ ]:
import schedule
import time

counter = 0

def job_that_runs_3_times():
    global counter
    counter += 1
    print(f"任务执行第 {counter} 次。")
    if counter >= 3:
        print("任务完成，将自行取消。")
        return schedule.CancelJob

schedule.every().second.do(job_that_runs_3_times)

while schedule.jobs: # 当还有任务在调度中时继续循环
    schedule.run_pending()
    time.sleep(1)

print("调度列表中已没有任务，程序结束。")

## `while True: pass` 和 `while True: time.sleep(1)`
| 特性 | `while True: pass` | `while True: time.sleep(1)` |
| :--- | :--- | :--- |
| **核心原理** | 忙碌等待 (Busy-Waiting) | 阻塞/睡眠 (Blocking/Sleeping) |
| **CPU占用率** | **极高** (通常接近100%占满一个核心) | **极低** (接近0%) |
| **能源消耗** | 高，导致设备发热，笔记本电脑耗电快 | 低，非常节能 |
| **对操作系统的影响** | 抢占CPU资源，可能导致其他应用变慢 | 将CPU资源让给其他应用，对系统友好 |
| **线程状态** | 持续处于**运行态 (Running)** | 在**运行态 (Running)** 和 **阻塞态 (Blocked/Sleeping)** 之间切换 |
| **与操作系统的交互** | 极少，仅在时间片切换时被动交互 | 频繁，通过**系统调用 (System Call)** 主动请求状态转换 |
| **适用场景** | 极少。仅用于某些底层同步或需要纳秒级精度的自旋锁（Spinlock）场景 | 绝大多数需要定时、轮询、等待的场景，如定时任务、心跳检测、服务器后台循环等 |
| **代码效率** | 效率极低，浪费计算资源 | 效率高，合理利用系统资源 |

# 代码设计

## 类继承关系

```mermaid
classDiagram
    %% 元类层次结构
    class asyncio.CancelledError
    class Job
    class Scheduler
    class CancelledError
    class AsyncJob
    class AsyncScheduler
    class ScheduleManager
    
    %% 继承关系
    asyncio.CancelledError <|-- CancelledError
    Job <|-- AsyncJob
    Scheduler <|-- AsyncScheduler
    ScheduleManager o-- AsyncScheduler : has
```
异步任务调度管理模块，主要用于：
1. 定期数据获取和更新（如股价数据、新闻数据等）
2. 策略定时执行（如每日交易信号生成）
3. 报告和监控任务（如风险监控、绩效报告）
4. 系统维护任务（如数据清理、缓存更新）

## class CancelledError(asyncio.CancelledError)

任务取消异常类，继承自 asyncio.CancelledError，用于标识任务取消事件。
```python
class CancelledError(asyncio.CancelledError):
    pass
```

例如：

In [ ]:
import vectorbt as vbt

async def risky_task():
    try:
        # 执行可能需要取消的任务
        await asyncio.sleep(10)
    except vbt.CancelledError:
        print("任务被用户取消")
        # 清理资源
        await asyncio.sleep(10)
        raise  # 重新抛出异常

## `class AsyncJob(Job)` 和 `class AsyncScheduler(Scheduler)`
`AsyncJob` 继承了 `schedule` 库中的任务类型 `Job`，用来支持异步任务。

`AsyncScheduler` 继承了 `schedule` 库中的调度器类型 `Scheduler`，用来支持异步调度。
- `.async_run_pending(self)` 类似于 `schedule` 的 `.run_pending()`，但是是异步执行。
- `.async_run_all(self, delay_seconds: int = 0)` 立即异步执行所有任务，`delay_seconds` 用于控制每个任务启动的间隔。

```python
class AsyncJob(Job):
    async def async_run(self) -> tp.Any:
        logger.info('Running job %s', self)
        ret = self.job_func()
        if inspect.isawaitable(ret):
            ret = await ret
        self.last_run = datetime.now()
        self._schedule_next_run()
        return ret

class AsyncScheduler(Scheduler):
    async def async_run_pending(self) -> None:
        runnable_jobs = (job for job in self.jobs if job.should_run)
        await asyncio.gather(*[self._async_run_job(job) for job in runnable_jobs])

    async def async_run_all(self, delay_seconds: int = 0) -> None:
        logger.info('Running *all* %i jobs with %is delay in-between',
                    len(self.jobs), delay_seconds)
        for job in self.jobs[:]:
            await self._async_run_job(job)
            await asyncio.sleep(delay_seconds)

    async def _async_run_job(self, job: AsyncJob) -> None:
        ret = await job.async_run()
        if isinstance(ret, CancelJob) or ret is CancelJob:
            self.cancel_job(job)

    def every(self, interval: int = 1) -> AsyncJob:
        job = AsyncJob(interval, self)
        return job
```

例如：

In [ ]:
import asyncio
import vectorbt as vbt

# 定义异步任务函数
async def fetch_stock_data(symbol):
    print(f"开始获取{symbol}的数据...")
    # 模拟异步API调用
    await asyncio.sleep(2)  # 模拟网络延迟
    print(f"完成获取{symbol}的数据")
    return f"{symbol}数据"

# 创建调度器并添加任务
scheduler = vbt.AsyncScheduler()

# 添加每分钟执行的数据获取任务
scheduler.every(1).do(fetch_stock_data, 'AAPL')
scheduler.every(1).do(fetch_stock_data, 'GOOG')
scheduler.every(5).do(fetch_stock_data, 'MSFT')

# 执行所有到期任务
async def main_loop():
    while True:
        
        await scheduler.async_run_pending()
        # 等待1秒后再次检查
        await asyncio.sleep(1)

# 或者使用 scheduler.async_run_all()，即执行所有任务
await scheduler.async_run_all()

await main_loop()

## ScheduleManager

### `__init__`
```python
def __init__(self, scheduler: tp.Optional[AsyncScheduler] = None) -> None:
    if scheduler is None:
        scheduler = AsyncScheduler()
    checks.assert_instance_of(scheduler, AsyncScheduler)

    # 私有属性，存储异步调度器实例
    self._scheduler = scheduler
    self._async_task = None
```

### every
生成设置了时间相关参数的 `AsyncJob` 对象。

参数：
- `*args`: 变长参数，按顺序可包含：
    - `interval` (int/timedelta): 执行间隔
        * 整数：表示时间间隔的数量
        * timedelta：表示具体的时间间隔
    - `unit` (str): 时间单位
        * 'second'/'seconds': 秒
        * 'minute'/'minutes': 分钟
        * 'hour'/'hours': 小时
        * 'day'/'days': 天
        * 'week'/'weeks': 周
    - `start_day` (str): 起始日期
        * 'monday'到'sunday': 具体的星期
    - `at` (str/time): 执行时间
        * 字符串格式：'HH:MM', 'HH:MM:SS', ':MM', ':MM:SS'
        * 时间对象：datetime.time实例
- `to` (int, optional): 随机间隔的上限，创建随机间隔任务
    - 与interval参数配合使用
    - 任务将在interval到to之间的随机时间执行
    - 用于避免多个任务同时执行造成的资源竞争
- `tags` (Iterable[Hashable], optional): 任务标签，用于分类和批量管理
    - 可以是单个标签或标签列表
    - 支持任何可哈希的对象作为标签
    - 用于任务分组、批量操作等

### start
启动一个无限循环来持续检查和执行到期的任务。
会阻塞当前线程，直到收到中断信号（如 Ctrl+C ）或程序异常退出。

```python
def start(self, sleep: int = 1) -> None:
    logger.info("Starting schedule manager with jobs %s", str(self.scheduler.jobs))
    try:
        while True:
            self.scheduler.run_pending()
            time.sleep(sleep)
    except (KeyboardInterrupt, asyncio.CancelledError):
        logger.info("Stopping schedule manager")
```

### async_start
相较于 `start`，区别是异步执行到期的任务，并且该函数自身也是异步，即不会阻塞当前线程。

```python
async def async_start(self, sleep: int = 1) -> None:
    logger.info("Starting schedule manager in the background with jobs %s", str(self.scheduler.jobs))
    logger.info("Jobs: %s", str(self.scheduler.jobs))
    try:
        while True:
            await self.scheduler.async_run_pending()
            await asyncio.sleep(sleep)
    except asyncio.CancelledError:
        logger.info("Stopping schedule manager")
```

### done_callback
`async_start` 完成后的回调函数，参考 `start_in_background`，实际使用时应当重新定义后重新设置给 `self.done_callback`。

```python
def done_callback(self, async_task: asyncio.Task) -> None:
    logger.info(async_task)
```

### start_in_background
创建异步任务 `async_start`，也就是不仅各任务不会阻塞，当前线程也不会阻塞。
- 该任务完成时，会自动调用回调函数 `self.done_callback`。
- 记录任务 `async_start` 到日志 `logger`。
- 保存 `async_start` 到 `self._async_task`。

```python
def start_in_background(self, **kwargs) -> None:
    async_task = asyncio.create_task(self.async_start(**kwargs))
    async_task.add_done_callback(self.done_callback)
    logger.info(async_task)
    self._async_task = async_task
```

### async_task_running
检查异步任务 `self.async_task` 是否存在且未完成。

```python
@property
def async_task_running(self) -> bool:
    return self.async_task is not None and not self.async_task.done()
```

### stop
停止异步任务 `self.async_task`。

```python
def stop(self) -> None:
    if self.async_task_running:
        self.async_task.cancel()
```

### 源码

```python
class ScheduleManager:

    units: tp.ClassVar[tp.Tuple[str, ...]] = (
        "second",
        "seconds",
        "minute",
        "minutes",
        "hour",
        "hours",
        "day",
        "days",
        "week",
        "weeks"
    )

    weekdays: tp.ClassVar[tp.Tuple[str, ...]] = (
        "monday",
        "tuesday",
        "wednesday",
        "thursday",
        "friday",
        "saturday",
        "sunday",
    )

    def __init__(self, scheduler: tp.Optional[AsyncScheduler] = None) -> None:
        if scheduler is None:
            scheduler = AsyncScheduler()
        checks.assert_instance_of(scheduler, AsyncScheduler)

        # 私有属性，存储异步调度器实例
        self._scheduler = scheduler
        self._async_task = None

    @property
    def scheduler(self) -> AsyncScheduler:
        return self._scheduler

    @property
    def async_task(self) -> tp.Optional[asyncio.Task]:
        return self._async_task

    def every(self, *args, to: tp.Optional[int] = None,
              tags: tp.Optional[tp.Iterable[tp.Hashable]] = None) -> AsyncJob:
        # Parse arguments
        interval = 1
        unit = None
        start_day = None
        at = None

        def _is_arg_interval(arg):
            return isinstance(arg, (int, timedelta))

        def _is_arg_unit(arg):
            return isinstance(arg, str) and arg in self.units

        def _is_arg_start_day(arg):
            return isinstance(arg, str) and arg in self.weekdays

        def _is_arg_at(arg):
            return (isinstance(arg, str) and ':' in arg) or isinstance(arg, dt_time)

        expected_args = ['interval', 'unit', 'start_day', 'at']
        for i, arg in enumerate(args):
            if 'interval' in expected_args and _is_arg_interval(arg):
                interval = arg
                expected_args = expected_args[expected_args.index('interval') + 1:]
                continue
            if 'unit' in expected_args and _is_arg_unit(arg):
                unit = arg
                expected_args = expected_args[expected_args.index('unit') + 1:]
                continue
            if 'start_day' in expected_args and _is_arg_start_day(arg):
                start_day = arg
                expected_args = expected_args[expected_args.index('start_day') + 1:]
                continue
            if 'at' in expected_args and _is_arg_at(arg):
                at = arg
                expected_args = expected_args[expected_args.index('at') + 1:]
                continue
            raise ValueError(f"Arg at index {i} is unexpected")

        if at is not None:
            if unit is None and start_day is None:
                unit = 'days'
        if unit is None and start_day is None:
            unit = 'seconds'

        job = self.scheduler.every(interval)
        if unit is not None:
            job = getattr(job, unit)
        if start_day is not None:
            job = getattr(job, start_day)
        if at is not None:
            if isinstance(at, dt_time):
                if job.unit == "days" or job.start_day:
                    if at.tzinfo is not None:
                        at = tzaware_to_naive_time(at, None)
                at = at.isoformat()
                if job.unit == "hours":
                    at = ':'.join(at.split(':')[1:])
                if job.unit == "minutes":
                    at = ':' + at.split(':')[2]
            job = job.at(at)
        if to is not None:
            job = job.to(to)
        if tags is not None:
            if not isinstance(tags, tuple):
                tags = (tags,)
            job = job.tag(*tags)

        return job

    def start(self, sleep: int = 1) -> None:
        logger.info("Starting schedule manager with jobs %s", str(self.scheduler.jobs))
        try:
            while True:
                self.scheduler.run_pending()
                time.sleep(sleep)
        except (KeyboardInterrupt, asyncio.CancelledError):
            logger.info("Stopping schedule manager")

    async def async_start(self, sleep: int = 1) -> None:
        logger.info("Starting schedule manager in the background with jobs %s", str(self.scheduler.jobs))
        logger.info("Jobs: %s", str(self.scheduler.jobs))
        try:
            while True:
                await self.scheduler.async_run_pending()
                await asyncio.sleep(sleep)
        except asyncio.CancelledError:
            logger.info("Stopping schedule manager")

    def done_callback(self, async_task: asyncio.Task) -> None:
        logger.info(async_task)

    def start_in_background(self, **kwargs) -> None:
        async_task = asyncio.create_task(self.async_start(**kwargs))
        async_task.add_done_callback(self.done_callback)
        logger.info(async_task)
        self._async_task = async_task

    @property
    def async_task_running(self) -> bool:
        return self.async_task is not None and not self.async_task.done()

    def stop(self) -> None:
        if self.async_task_running:
            self.async_task.cancel()
```